In [12]:
from google.cloud import bigquery
from google.cloud.bigquery import job
from google.cloud.bigquery import SchemaField
import pandas as pd
import datetime as dt
import numpy as np
import re
import os
from tqdm import tqdm
import pandas as pd
import copy

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
new_table = str(input('incomming table : '))
tg_table = str(input('target table : '))

new_table_ref = f'ballosodeuk.{dataset_id}.{new_table}'
tg_table_ref = f'ballosodeuk.{dataset_id}.{tg_table}'

print("target : ",new_table_ref)
print("to : ",tg_table_ref)

client = bigquery.Client()

target :  ballosodeuk.airbridge_mart.test_append
to :  ballosodeuk.airbridge_mart.app_keyfeatures


### 스키마 업데이트 시 SQL 문의 fin_table 순서가 바뀌지 않도록 주의

In [30]:
def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    # return {field.name: field for field in table.schema}
    return table.schema

# 기존 스키마 추출
def convert_schema_fields(schema):
    def convert_field(field):
        if field.field_type == 'RECORD':
            return {
                "type": field.field_type,
                "mode": field.mode,
                "fields": {subfield.name: convert_field(subfield) for subfield in field.fields}
            }
        elif field.field_type == 'STRING' and field.mode == 'REPEATED':
            return {
                "type": field.field_type,
                "mode": field.mode,
                "fields": {subfield.name: convert_field(subfield) for subfield in field.fields} if field.fields else {}
            }
        else:
            return field.field_type

    return {field.name: convert_field(field) for field in schema}

def order_schema(schema, order):
    ordered_schema = {}
    for key in order:
        if key in schema:
            ordered_schema[key] = schema[key]
    return ordered_schema

def convert_to_schema_fields(schema):
    """스키마 딕셔너리를 SchemaField 객체로 변환."""
    fields = []
    for name, dtype in schema.items():
        if isinstance(dtype, dict):
            subfields = convert_to_schema_fields(dtype.get("fields", {}))
            mode = dtype.get("mode", "NULLABLE")
            fields.append(SchemaField(name, dtype["type"], mode=mode, fields=subfields))
        else:
            fields.append(SchemaField(name, dtype))
    return fields


def update_table_schema(table_ref, merged_schema):
    """테이블 스키마에 새 열을 추가합니다."""
    table = client.get_table(table_ref)
    table.schema = merged_schema
    client.update_table(table, ["schema"])
    print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {merged_schema}")

# def reorder_schema(schema, desired_order):
#     """스키마를 원하는 순서로 재정렬합니다."""
#     field_dict = {field.name: field for field in schema}
#     new_schema = [field_dict[field_name] for field_name in desired_order]
#     return new_schema

### 데이터 추가
새로운 임시테이블로 해야함

In [43]:
## 신규 데이터의 임시 테이블 생성

query_name = "0716_정적쿼리앱"
# SQL 파일 읽기
# file = './query/쿠팡0403_0509.sql'
file = f'../query/{query_name}.sql'
with open(file, 'r') as file:
    query = file.read()
    
job_config = bigquery.QueryJobConfig()
query_job = client.query(query, job_config=job_config)
query_job.result()

### 뉴스키마는 생성된 임시 테이블일것. 순서는 기존에 맞춰서 추가해야함.

In [31]:
new_schema = get_current_schema(new_table_ref)
current_schema = get_current_schema(tg_table_ref)

cur_dict = convert_schema_fields(current_schema)
new_dict = convert_schema_fields(new_schema)

result_dict = order_schema(new_dict, cur_dict)
scheme = convert_to_schema_fields(result_dict)
update_table_schema(new_table_ref, scheme)
update_table_schema(tg_table_ref, scheme)

테이블 ballosodeuk.airbridge_mart.test_append의 스키마가 업데이트되었습니다: [SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('User_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Channel', 'STRING', 'NULLABLE', None, (), None), SchemaField('Campaign_ID', 'STRING', 'NULLABLE', None, (), None), Schema

In [34]:
update_table_schema(new_table_ref, scheme)

테이블 ballosodeuk.airbridge_mart.test_append의 스키마가 업데이트되었습니다: [SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('User_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Channel', 'STRING', 'NULLABLE', None, (), None), SchemaField('Campaign_ID', 'STRING', 'NULLABLE', None, (), None), Schema

### 안될때 아래

In [38]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
new_table = str(input('incomming table : '))
tg_table = str(input('target table : '))

new_table_ref = f'ballosodeuk.{dataset_id}.{new_table}'
tg_table_ref = f'ballosodeuk.{dataset_id}.{tg_table}'

print("target : ", new_table_ref)
print("to : ", tg_table_ref)

client = bigquery.Client()

def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    return table.schema

def reorder_schema(schema, desired_order):
    """스키마를 원하는 순서로 재정렬합니다."""
    field_dict = {field.name: field for field in schema}
    new_schema = []
    for field_name in desired_order:
        if field_name in field_dict:
            field = field_dict[field_name]
            if field.field_type == 'RECORD':
                # Nested fields: reorder recursively
                nested_schema = reorder_schema(field.fields, [f.name for f in field.fields])
                new_field = SchemaField(name=field.name, field_type=field.field_type, mode=field.mode, fields=nested_schema)
            else:
                new_field = field
            new_schema.append(new_field)
    return new_schema

def update_table_schema(table_ref, new_schema):
    """테이블 스키마를 업데이트합니다."""
    table = client.get_table(table_ref)
    table.schema = new_schema
    client.update_table(table, ["schema"])
    print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_schema}")

# 기존 스키마 가져오기
current_schema = get_current_schema(tg_table_ref)

# 원하는 순서 정의
desired_order = [
    'Event_Date', 'Airbridge_Device_ID', 'Airbridge_Device_ID_Type', 'User_ID', 'Platform',
    'i', 'Event_Category', 'Event_Value_Total', 'Event_Count_Total', 'e'
    # 나머지 필드들도 원하는 순서대로 추가
]

# 스키마 재정렬
new_schema = reorder_schema(current_schema, desired_order)

# 테이블 스키마 업데이트
update_table_schema(new_table_ref, new_schema)


target :  ballosodeuk.airbridge_mart.test_append
to :  ballosodeuk.airbridge_mart.app_keyfeatures
테이블 ballosodeuk.airbridge_mart.test_append의 스키마가 업데이트되었습니다: [SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('User_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Channel', 'STRING', 'NU

In [40]:
from google.cloud import bigquery
from google.cloud.bigquery import SchemaField

PROJCECT = 'ballosodeuk'
bq = bigquery.Client(project=PROJCECT)
dataset_id = 'airbridge_mart'
new_table = str(input('incomming table : '))
tg_table = str(input('target table : '))

new_table_ref = f'ballosodeuk.{dataset_id}.{new_table}'
tg_table_ref = f'ballosodeuk.{dataset_id}.{tg_table}'

print("target : ", new_table_ref)
print("to : ", tg_table_ref)

client = bigquery.Client()

def get_current_schema(table_ref):
    """기존 테이블의 스키마를 가져옵니다."""
    table = client.get_table(table_ref)
    return table.schema

def convert_schema_to_dict(schema):
    """SchemaField 객체를 딕셔너리로 변환."""
    def convert_field(field):
        if field.field_type == 'RECORD':
            return {
                "type": field.field_type,
                "mode": field.mode,
                "fields": convert_schema_to_dict(field.fields)
            }
        else:
            return {
                "type": field.field_type,
                "mode": field.mode
            }
    return {field.name: convert_field(field) for field in schema}

def reorder_schema_dict(schema_dict, desired_order):
    """스키마 딕셔너리를 원하는 순서로 재정렬."""
    ordered_schema = {}
    for key in desired_order:
        if key in schema_dict:
            print(key)
            ordered_schema[key] = schema_dict[key]
    return ordered_schema

def convert_dict_to_schema(schema_dict):
    """딕셔너리를 SchemaField 객체로 변환."""
    def convert_field(name, field_info):
        if field_info["type"] == "RECORD":
            return SchemaField(name, field_info["type"], mode=field_info["mode"], fields=convert_dict_to_schema(field_info["fields"]))
        else:
            return SchemaField(name, field_info["type"], mode=field_info["mode"])
    return [convert_field(name, field_info) for name, field_info in schema_dict.items()]

def update_table_schema(table_ref, new_schema):
    """테이블 스키마를 업데이트합니다."""
    table = client.get_table(table_ref)
    table.schema = new_schema
    client.update_table(table, ["schema"])
    print(f"테이블 {table_ref}의 스키마가 업데이트되었습니다: {new_schema}")

# 기존 스키마 가져오기
current_schema = get_current_schema(tg_table_ref)

# 스키마를 딕셔너리로 변환
current_schema_dict = convert_schema_to_dict(current_schema)

# 원하는 순서 정의
desired_order = [
    'Event_Date', 'Airbridge_Device_ID', 'Airbridge_Device_ID_Type', 'User_ID', 'Platform',
    'i', 'Event_Category', 'Event_Value_Total', 'Event_Count_Total', 'e'
    # 나머지 필드들도 원하는 순서대로 추가
]

# 스키마 딕셔너리를 재정렬
ordered_schema_dict = reorder_schema_dict(current_schema_dict, desired_order)

# 스키마 딕셔너리를 SchemaField 객체로 변환
new_schema = convert_dict_to_schema(ordered_schema_dict)

# 테이블 스키마 업데이트
update_table_schema(tg_table_ref, new_schema)
new_schema


target :  ballosodeuk.airbridge_mart.test_append
to :  ballosodeuk.airbridge_mart.app_keyfeatures
Event_Date
Airbridge_Device_ID
Airbridge_Device_ID_Type
User_ID
Platform
i
Event_Category
Event_Value_Total
Event_Count_Total
e


[SchemaField('Event_Date', 'DATE', 'NULLABLE', None, (), None),
 SchemaField('Airbridge_Device_ID', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('Airbridge_Device_ID_Type', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('User_ID', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None),
 SchemaField('i', 'RECORD', 'NULLABLE', None, (SchemaField('Device_Model', 'STRING', 'NULLABLE', None, (), None), SchemaField('Device_Type', 'STRING', 'NULLABLE', None, (), None), SchemaField('Platform', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Country_Code', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_Subdivision', 'STRING', 'NULLABLE', None, (), None), SchemaField('Client_IP_City', 'STRING', 'NULLABLE', None, (), None), SchemaField('Channel', 'STRING', 'NULLABLE', None, (), None), SchemaField('Campaign_ID', 'STRING', 'NULLABLE', None, (), None), SchemaField('Ad_Group_ID', 'STRING', 'NULLABLE', None, (), No

In [ ]:
update_table_schema(tg_table_ref, new_schema)